In [83]:
import requests as rq
import json
import pandas as pd
pd.options.plotting.backend = "plotly"

In [2]:
# Get API key
def get_key():
    f = open("/home/vikas/Documents/CG_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

## Check API status

In [3]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [5]:
use_demo = {"accept": "application/json"}

In [6]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_key()          
}

In [16]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

## NFTs

#### Demo API

In [17]:
nft_params = {
            "order": "floor_price_native_desc",
            "per_page": 10,
            "page": 1
}

In [18]:
nft_list_response = get_response("/nfts/list", use_demo, nft_params, PUB_URL)

In [19]:
nft_list_response;

In [20]:
df_nft = pd.DataFrame(nft_list_response)

In [21]:
df_nft

,id,contract_address,name,asset_platform_id,symbol
0,gaia-supernova-deprecated,0x89a18abab20aab069feb7cab20517630ee7c1626,Gaia Supernova,klay-token,GAIA
1,gaia-genesis-deprecated,0xbb915237d8b46dcdfe813c914bf98708e0dad84a,Gaia Kronos,klay-token,GAIA
2,burned-202209,0xc6e5ba514dc2ec6a556dbb95d3002cc1070ad7b9,Dicast Hero Badge SuperRare Pass,klay-token,DHBSRP
3,gaia-protocol-genesis-klaytn,0xe9a10bb97ddb4bcd7677393405b4b769273cef3c,Gaia Protocol Genesis,klay-token,GAIA
4,syltare-dawn-of-east-klaytn,0xdf104df1b238dcd1be2d160cea47893e03b3f9b9,SYL - Dawn of East,klay-token,SYL
5,pixels-pets,0xb806028b6ebc35926442770a8a8a7aeab6e2ce5c,Pixels Pets,ronin,PIXELS-PETS
6,footage-by-sportium,0x839e43c5a768890f5d35ecdf0fb0b356529a80b5,FOOTAGE BY SPORTIUM,klay-token,FOOTAGE
7,syltare-frozen,0xe2dd65c215089dda1695ae96cce77f301a1750e5,SYL - Dawn of East,klay-token,SYL
8,the-crop-collective,0xe5863d4a57b1e2d7031f7969fee4f56bb04c0f17,The Crop Collective,ethereum,$TCC
9,claynosaurz-sardinhas,,Claynosaurz: Sardinhas,solana,CLAYSARDINHAS


In [22]:
df_nft.id[4]

'syltare-dawn-of-east-klaytn'

In [23]:
def get_nft_current_data(index, df_nft, use_demo = use_demo, PUB_URL = PUB_URL):
    nft_url = f"/nfts/{df_nft.id[index]}"
    nft_current_response = get_response(nft_url, use_demo, nft_params, PUB_URL)
    
    new_data = dict(id = nft_current_response["id"], 
                    name = nft_current_response["name"],
                    symbol = nft_current_response["symbol"],
                    floor_price_usd = nft_current_response["floor_price"]["usd"])
    
    df_nft_current = pd.DataFrame([new_data])
    return df_nft_current

In [24]:
df_nft_current = get_nft_current_data(5, df_nft)

In [25]:
df_nft_current

,id,name,symbol,floor_price_usd
0,pixels-pets,Pixels Pets,PIXELS-PETS,1118.74


#### Pro API

In [26]:
page_params = {
            "per_page": 100,
            "page": 1,
            "order": "h24_volume_usd_desc"    
}

In [27]:
nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL)

In [41]:
nft_markets[10];

In [37]:
def get_nft_markets(num_entries, order_by):
    
    page_params = {
                    "per_page": num_entries,
                    "page": 1,
                    "order": order_by    
                  }
    
    nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL) 
    nft_all = []

    for i in range(len(nft_markets)):
        nft_select = nft_markets[i]
        temp_dict = dict(
                         id = nft_select["id"],
                         asset_platform_id  = nft_select["asset_platform_id"],
                         floor_price_native = nft_select["floor_price"]["native_currency"],
                         floor_price_usd    = nft_select["floor_price"]["usd"],
                         market_cap_usd     = nft_select["market_cap"]["usd"],
                         volume_24h_usd     = nft_select["volume_24h"]["usd"]
                        )
        nft_all.append(temp_dict)
            
    return pd.DataFrame(nft_all)                     


In [38]:
df_nft_markets = get_nft_markets(10, "h24_volume_usd_desc")

In [68]:
df_nft_markets.id[7]

'degods'

In [54]:
hist_params = {"days": 2}

In [55]:
nft_hist = get_response(f"/nfts/{df_nft_markets.id[4]}/market_chart", use_pro, hist_params, PRO_URL)

In [63]:
nft_hist["floor_price_usd"][1]

[1699041610000, 3.633354149229174]

In [77]:
def get_nft_hist(nft_id, num_days):
    hist_params = {"days": num_days}
    hist_url = f"/nfts/{nft_id}/market_chart"
    
    nft_hist = get_response(hist_url, use_pro, hist_params, PRO_URL)
    all_time, all_floor_price_usd = [], []    

    floor_price_usd = nft_hist["floor_price_usd"]

    for i in range(len(floor_price_usd)):
        time = floor_price_usd[i][0]
        all_time.append(time)
        
        price = floor_price_usd[i][1]
        all_floor_price_usd.append(price)

    df_hist = pd.DataFrame(list(zip(all_time, all_floor_price_usd)),
                           columns =['Time', 'Price_usd'])

    df_hist["Time"] = pd.to_datetime(df_hist["Time"], unit = "ms")
    df_hist.set_index("Time", inplace = True)

    return df_hist    
    

In [78]:
df_hist = get_nft_hist(df_nft_markets.id[7], 10)

In [79]:
df_hist

,Price_usd
Time,
2023-10-26 20:10:12,6008.634598
2023-10-26 20:15:08,6008.634598
2023-10-26 20:20:09,6008.634598
2023-10-26 20:25:10,6008.634598
2023-10-26 20:30:21,5910.750663
...,...
2023-11-05 19:45:11,6058.897430
2023-11-05 19:50:09,6058.897430
2023-11-05 19:55:10,6058.897430


## Plot data

In [84]:
fig1 = df_hist.plot()

In [85]:
fig1.show()

In [86]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()